In [3]:
import pandas as pd

In [86]:
dates=pd.date_range(start='2020-11-08 00:00:00', end='2021-01-27 14:00:00',freq='1H')
dates_url = dates.strftime('%Y%m%d_%H%M%S')
columns=['Date','Rank','Nat/Sail','Skipper/Crew','Hour FR','Latitude','Longitude','Heading_30min','Speed_30min','VMG_30min','Distance_30min','Heading_lrt','Speed_lrt','VMG_lrt','Distance_lrt','Heading_24h','Speed_24h','VMG_24h','Distance_24h','DTF','DTL']
df_read = pd.DataFrame()
for idx,value in enumerate(dates_url):
    
    url="https://www.vendeeglobe.org/download-race-data/vendeeglobe_"+value+".xlsx"
    try:
        csv=pd.read_excel(url,skiprows=4,names=columns,skipfooter=4)
        csv['Date']=dates[idx]
        df_read = df_read.append(csv, ignore_index=True)
    except:
        continue

In [87]:
df_read.to_csv('vendeeglobe_race_date.csv',index=False)

In [322]:
df=pd.read_csv('vendeeglobe_race_date.csv')

In [323]:
df.shape

(15906, 21)

In [324]:
#df.head(10)

In [325]:
#df['Skipper']=df['Skipper/Crew'].apply(lambda x: x.split('\n')[0])
#df['Crew']=df['Skipper/Crew'].apply(lambda x: x.split('\n')[1])

In [326]:
df[['Skipper','Crew']]=df['Skipper/Crew'].str.extract('(.*)\n(.*)')
df['Sail']=df['Nat/Sail'].str.extract('(\d+)')
df['Nat']=df['Nat/Sail'].apply(lambda x: x.split('\n')[1].split(' ')[0])

In [327]:
#Fill nan values for NL Sails at the starting date 

change_columns=['Hour FR','Latitude','Longitude','Heading_30min','Speed_30min','VMG_30min','Distance_30min','Heading_lrt','Speed_lrt','VMG_lrt','Distance_lrt','Heading_24h','Speed_24h','VMG_24h','Distance_24h','DTF','DTL']
df.loc[(df['Rank']=='NL')&(df['Date']=="2020-11-08 14:00:00"),change_columns]=df.loc[0,change_columns].values

In [328]:
#Sort by date and sail to fillna backward
df = df.sort_values(by=['Sail','Date'], ascending=(True,False))
df=df.fillna(method='bfill')


In [329]:
final_df.isna().sum()

Date              0
Rank              0
Nat/Sail          0
Skipper/Crew      0
Hour FR           0
Latitude          0
Longitude         0
Heading_30min     0
Speed_30min       0
VMG_30min         0
Distance_30min    0
Heading_lrt       0
Speed_lrt         0
VMG_lrt           0
Distance_lrt      0
Heading_24h       0
Speed_24h         0
VMG_24h           0
Distance_24h      0
DTF               0
DTL               0
Skipper           0
Crew              0
Sail              0
Nat               0
dtype: int64

In [330]:
df.loc[0,['Sail','Longitude']].values

array(['18', "01°50.48'W"], dtype=object)

In [331]:
final_df.loc[(final_df['Rank']=='NL')]

,Date,Rank,Nat/Sail,Skipper/Crew,Hour FR,Latitude,Longitude,Heading_30min,Speed_30min,VMG_30min,...,Heading_24h,Speed_24h,VMG_24h,Distance_24h,DTF,DTL,Skipper,Crew,Sail,Nat
98,2020-11-08 17:00:00,NL,\nFRA 14,Arnaud Boissieres\nLa Mie Câline - Artisans Ar...,14:30 FR\n,47°43.38'N,07°58.81'W,98°,18.4 kts,18.3 kts,...,91°,18.6 kts,18.6 kts,447.2 nm,264.3 nm,183.7 nm,Arnaud Boissieres,La Mie Câline - Artisans Artipôle,14,FRA
65,2020-11-08 15:00:00,NL,\nFRA 14,Arnaud Boissieres\nLa Mie Câline - Artisans Ar...,14:30 FR\n,47°43.38'N,07°58.81'W,98°,18.4 kts,18.3 kts,...,91°,18.6 kts,18.6 kts,447.2 nm,264.3 nm,183.7 nm,Arnaud Boissieres,La Mie Câline - Artisans Artipôle,14,FRA
32,2020-11-08 14:00:00,NL,\nFRA 14,Arnaud Boissieres\nLa Mie Câline - Artisans Ar...,14:30 FR\n,47°43.38'N,07°58.81'W,98°,18.4 kts,18.3 kts,...,91°,18.6 kts,18.6 kts,447.2 nm,264.3 nm,183.7 nm,Arnaud Boissieres,La Mie Câline - Artisans Artipôle,14,FRA
31,2020-11-08 14:00:00,NL,\nFRA 79,Charlie Dalin\nAPIVIA,14:30 FR\n,09°29.21'N,34°26.52'W,342°,15.5 kts,10.6 kts,...,347°,13.9 kts,11.0 kts,334.3 nm,2788.4 nm,2707.8 nm,Charlie Dalin,APIVIA,79,FRA
4552,2020-12-01 08:00:00,NL,\nFRA 85,Kevin Escoffier\nPRB,15:00 FR\n-810min,40°56.67'S,09°16.33'E,115°,9.7 kts,9.6 kts,...,102°,18.0 kts,17.8 kts,432.6 nm,17957.7 nm,470.5 nm,Kevin Escoffier,PRB,85,FRA


In [332]:
def degreeToDec(string):
    direction=string[-1]
    sec=int(string.split(".")[1].split("'")[0])
    mins=int(string.split(".")[0].split("°")[1])
    degree=int(string.split("°")[0])
    dec = degree + (mins/60) + (sec/3600)
    if direction=='S'or direction=='W':
        dec=(-dec)
    return dec

In [333]:
a="46°24.46'S"
g=degreeToDec(a)
type(g)

float

In [334]:
df['Latitude']=df['Latitude'].apply(lambda x: degreeToDec(x))
df['Longitude']=df['Longitude'].apply(lambda x: degreeToDec(x))

In [337]:
df

,Date,Rank,Nat/Sail,Skipper/Crew,Hour FR,Latitude,Longitude,Heading_30min,Speed_30min,VMG_30min,...,Heading_24h,Speed_24h,VMG_24h,Distance_24h,DTF,DTL,Skipper,Crew,Sail,Nat
15880,2021-01-27 14:00:00,8,\nFRA 01,Jean Le Cam\nYes we Cam !,14:30 FR\n,44.708889,-13.708611,70°,18.1 kts,18.0 kts,...,59°,17.1 kts,17.1 kts,410.4 nm,511.6 nm,431.1 nm,Jean Le Cam,Yes we Cam !,01,FRA
15847,2021-01-27 11:00:00,8,\nFRA 01,Jean Le Cam\nYes we Cam !,11:30 FR\n,44.336944,-14.940556,55°,15.1 kts,14.4 kts,...,58°,16.6 kts,16.6 kts,399.6 nm,568.3 nm,439.4 nm,Jean Le Cam,Yes we Cam !,01,FRA
15814,2021-01-27 08:00:00,8,\nFRA 01,Jean Le Cam\nYes we Cam !,08:30 FR\n,43.808056,-15.891389,55°,17.8 kts,17.2 kts,...,58°,16.4 kts,16.4 kts,393.9 nm,617.6 nm,432.8 nm,Jean Le Cam,Yes we Cam !,01,FRA
15781,2021-01-27 04:00:00,8,\nFRA 01,Jean Le Cam\nYes we Cam !,04:30 FR\n,43.111111,-17.127500,56°,17.8 kts,17.4 kts,...,59°,16.1 kts,16.1 kts,386.3 nm,682.7 nm,421.7 nm,Jean Le Cam,Yes we Cam !,01,FRA
15748,2021-01-26 21:00:00,8,\nFRA 01,Jean Le Cam\nYes we Cam !,21:30 FR\n,41.956111,-19.313611,56°,16.9 kts,16.8 kts,...,62°,15.0 kts,15.0 kts,360.1 nm,799.5 nm,461.7 nm,Jean Le Cam,Yes we Cam !,01,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,2020-11-09 04:00:00,19,\nGBR 99,Alex Thomson\nHugo Boss,05:30 FR\n,46.490833,-6.891389,290°,12.3 kts,1.6 kts,...,268°,9.0 kts,7.9 kts,210.5 nm,24147.4 nm,14.3 nm,Alex Thomson,Hugo Boss,99,GBR
104,2020-11-08 21:00:00,6,\nGBR 99,Alex Thomson\nHugo Boss,22:30 FR\n,46.151389,-4.669167,266°,5.1 kts,4.1 kts,...,259°,5.2 kts,4.9 kts,121.3 nm,24188.3 nm,9.5 nm,Alex Thomson,Hugo Boss,99,GBR
69,2020-11-08 17:00:00,4,\nGBR 99,Alex Thomson\nHugo Boss,18:30 FR\n,46.259167,-3.326111,265°,20.2 kts,17.6 kts,...,256°,2.8 kts,2.7 kts,64.9 nm,24237.0 nm,2.0 nm,Alex Thomson,Hugo Boss,99,GBR
35,2020-11-08 15:00:00,3,\nGBR 99,Alex Thomson\nHugo Boss,17:00 FR\n,46.318889,-2.535000,263°,22.8 kts,20.8 kts,...,249°,1.4 kts,1.3 kts,32.7 nm,24266.3 nm,1.2 nm,Alex Thomson,Hugo Boss,99,GBR
